In [3]:
import cv2
from mtcnn.mtcnn import MTCNN
import os

cap = cv2.VideoCapture(0)
counter = 0
num_samples = 300

SE171070_TRAIN_DIR = os.path.join("data", "train", "SE171070")
SE150905_TRAIN_DIR = os.path.join("data", "train", "SE150905")

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    detector = MTCNN()
    faces = detector.detect_faces(gray)

    min_conf = 0.9

    for face in faces:
        if face["confidence"] >= min_conf:
            x, y, w, h = face["box"]
            # cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            roi = frame[y - 20 : y + h + 20, x - 20 : x + w + 20]
            while counter <= 300:
                imgname = os.path.join(SE150905_TRAIN_DIR, "SE150905_{}.jpg".format(i))
                cv2.imwrite(imgname, roi)
                counter += 1
    cv2.imshow("My face detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 138ms/step


### Augmentation

In [1]:
import albumentations as alb
import cv2 
import os
from keras_preprocessing.image import ImageDataGenerator

In [2]:
TRAINING_DIR = os.path.join('data', 'train')

training_augment = ImageDataGenerator(
    rescale= 1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

TESTING_DIR = os.path.join('data', 'test')

testing_augment = ImageDataGenerator(
    rescale=1./255
)

train_generator = training_augment.flow_from_directory(
    TRAINING_DIR,
    target_size=(150, 150),
    class_mode='categorical'
)

test_generator = testing_augment.flow_from_directory(
    TESTING_DIR,
    target_size=(150,150),
    class_mode='categorical'
)

Found 602 images belonging to 3 classes.
Found 211 images belonging to 3 classes.


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Dense, GlobalMaxPooling2D

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
face_recognition = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    # The second convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [6]:
face_recognition.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       147584    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

In [7]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=40000,
    decay_rate=0.1,
    staircase=True
)

In [8]:
face_recognition.compile(
    optimizer=tf.keras.optimizers.experimental.AdamW(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [9]:
history = face_recognition.fit(
    train_generator, epochs=10, validation_data=test_generator, verbose=1, batch_size=8
)

face_recognition.save("face_recognition.h5")

Epoch 1/10


InternalError: Graph execution error:

Detected at node 'StatefulPartitionedCall_10' defined at (most recent call last):
    File "c:\ProgramData\miniconda3\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\ProgramData\miniconda3\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\ProgramData\miniconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\ProgramData\miniconda3\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "c:\ProgramData\miniconda3\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "c:\ProgramData\miniconda3\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\ProgramData\miniconda3\lib\asyncio\base_events.py", line 603, in run_forever
      self._run_once(